# 3. InceptionResNetV2_SE_Predict

## Run name

In [1]:
import time

project_name = 'inclusive-images-challenge'
step_name = '3_InceptionResNetV2_SE_Predict'
time_str = time.strftime("%Y%m%d-%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)

time0 = time.time()

run_name: inclusive-images-challenge_3_InceptionResNetV2_SE_Predict_20181026-132138


## Import pkgs

In [2]:
import os
import random
import time
import re
from tqdm import tqdm
import multiprocessing
from tqdm import tqdm_notebook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score, jaccard_similarity_score

cpu_amount = multiprocessing.cpu_count()
random_num = np.random.randint(0, 9999)
random_num_str = '%04d' % random_num

print('cpu_amount: %s' % (cpu_amount - 1))
print('random_num: %s' % random_num_str)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/font_manager.py:278: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


cpu_amount: 1
random_num: 7912


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Project folders

In [3]:
cwd = os.getcwd()
feature_folder = os.path.join(cwd, 'feature')
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')
log_folder = os.path.join(cwd, 'log')

cwd = os.getcwd()
feature_folder = os.path.join(cwd, 'feature')
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')
log_folder = os.path.join(cwd, 'log')

print(feature_folder)
print(input_folder)
print(output_folder)
print(model_folder)
print(log_folder)
train_folder = os.path.join(input_folder, 'train')
validation_folder = os.path.join(input_folder, 'validation')
test_folder = os.path.join(input_folder, 'test')
stage_1_test_images_folder = os.path.join(input_folder, 'stage_1_test_images')
sl_train_folder = os.path.join(input_folder, 'sl_train')
sl_val_folder = os.path.join(input_folder, 'sl_val')
sl_test_folder = os.path.join(input_folder, 'sl_test')

print(train_folder)
print(validation_folder)
print(test_folder)
print(stage_1_test_images_folder)
print(sl_train_folder)
print(sl_val_folder)
print(sl_test_folder)

class_descriptions_csv = os.path.join(input_folder, 'class-descriptions.csv')
class_trainable_csv = os.path.join(input_folder, 'classes-trainable.csv')
stage_1_attributions_csv = os.path.join(input_folder, 'stage_1_attributions.csv')
stage_1_sample_submission_csv = os.path.join(input_folder, 'stage_1_sample_submission.csv')
train_bounding_boxes_csv = os.path.join(input_folder, 'train_bounding_boxes.csv')
train_human_labels_csv = os.path.join(input_folder, 'train_human_labels.csv')
train_machine_labels_csv = os.path.join(input_folder, 'train_machine_labels.csv')
tuning_labels_csv = os.path.join(input_folder, 'tuning_labels.csv')

print(class_descriptions_csv)
print(class_trainable_csv)
print(stage_1_attributions_csv)
print(stage_1_sample_submission_csv)
print(train_bounding_boxes_csv)
print(train_human_labels_csv)
print(train_machine_labels_csv)
print(tuning_labels_csv)

/data/kaggle-Inclusive-Images/feature
/data/kaggle-Inclusive-Images/input
/data/kaggle-Inclusive-Images/output
/data/kaggle-Inclusive-Images/model
/data/kaggle-Inclusive-Images/log
/data/kaggle-Inclusive-Images/input/train
/data/kaggle-Inclusive-Images/input/validation
/data/kaggle-Inclusive-Images/input/test
/data/kaggle-Inclusive-Images/input/stage_1_test_images
/data/kaggle-Inclusive-Images/input/sl_train
/data/kaggle-Inclusive-Images/input/sl_val
/data/kaggle-Inclusive-Images/input/sl_test
/data/kaggle-Inclusive-Images/input/class-descriptions.csv
/data/kaggle-Inclusive-Images/input/classes-trainable.csv
/data/kaggle-Inclusive-Images/input/stage_1_attributions.csv
/data/kaggle-Inclusive-Images/input/stage_1_sample_submission.csv
/data/kaggle-Inclusive-Images/input/train_bounding_boxes.csv
/data/kaggle-Inclusive-Images/input/train_human_labels.csv
/data/kaggle-Inclusive-Images/input/train_machine_labels.csv
/data/kaggle-Inclusive-Images/input/tuning_labels.csv


In [4]:
print(run_name)
print('Time elapsed: %.2f s' % (time.time() - time0))

inclusive-images-challenge_3_InceptionResNetV2_SE_Predict_20181026-132138
Time elapsed: 51.60 s
